In [18]:
import pandas as pd
import os

In [19]:
df = pd.read_csv('/Users/danielepica/Documents/GitHub/recommender_system_llm/src/grouped_book_recommendation_100.csv')

In [20]:
df.head()

,User_id,books_info
0,ASXVN70L40AMR,"title:Wuthering Heights (New Windmill), author..."
1,AOKVU6KSH4OB6,"title:THE THREE FAT MEN A Novel For Children, ..."
2,A84YWOJ6K2CUP,title:Dream Brother: The Lives and Music of Je...
3,A1P2BNZCJZ0IOJ,title:The Five Love Languages: The Secret to L...
4,A3JC9FG56Z8GSO,title:A Gown of Spanish Lace (Women of the Wes...


Ogni riga del dataset (che rappresenta un utente) sarà trasformata in un embedding che rappresenta 
la sua lista di libri e recensioni. Questi embeddings saranno utilizzati per calcolare la somiglianza tra utenti.

In [4]:
import pandas as pd
from langchain.vectorstores import Chroma
from langchain.embeddings import OllamaEmbeddings
import os


# Inizializza il modello di embedding
embeddings = OllamaEmbeddings(model='mxbai-embed-large', show_progress=True)

# Inizializza Chroma
persist_directory = 'db_user_based'
if os.path.exists(persist_directory):
    import shutil
    shutil.rmtree(persist_directory)

vectorstore = Chroma(embedding_function=embeddings, persist_directory=persist_directory)

# Crea embeddings per ogni utente basato sul campo 'book_info'
for idx, row in df.iterrows():
    document_text = row['books_info']
    user_id = row['User_id']
    
    # Aggiungi il documento alla base di dati vettoriale Chroma
    vectorstore.add_texts(texts=[document_text], metadatas=[{'user_id': user_id}])

# Persistenza dei dati
vectorstore.persist() #potrei toglierlo non è più necessario


OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 32.75it/s]
/opt/anaconda3/envs/LangChain/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


Ora che abbiamo creato e salvato gli embeddings, possiamo utilizzare il modello per trovare utenti simili e fare raccomandazioni.

In [14]:
def find_similar_users(user_id, query, vectorstore, top_k=5):
    # Recupera i documenti/metadati per l'utente specifico
    similar_docs = vectorstore.similarity_search_with_score(query, k=5)

    # Filtra i documenti simili in base ai metadati
    user_docs = [doc for doc in similar_docs if doc[0].metadata['user_id'] != user_id]

    return user_docs
    

# Esempio di utilizzo
user_id = "A1P2BNZCJZ0IOJ"
query = "page_content=title:The Five Love Languages: The Secret to Love that Lasts, authors:['Gary Chapman'], categories:['Family & Relationships'], review/score:5.0,title:The Happiest Baby on the Block: The New Way to Calm Crying and Help Your Newborn Baby Sleep Longer, authors:['Harvey Karp'], categories:['FAMILY & RELATIONSHIPS'], review/score:5.0"

similar_users = find_similar_users(user_id, query, vectorstore)

for user in similar_users:
    print(f"User ID: {user[0].metadata['user_id']}, Similarity Score: {user[1]}")
    print(user[0].page_content)



OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  9.30it/s]


AttributeError: 'tuple' object has no attribute 'metadata'

In [17]:
similar_docs = vectorstore.similarity_search_with_score(query, k=5)
print(similar_docs)
for doc in similar_docs:
    print(doc[0].metadata['user_id'])

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  7.41it/s]

[(Document(metadata={'user_id': 'A1P2BNZCJZ0IOJ'}, page_content="title:The Five Love Languages: The Secret to Love that Lasts, authors:['Gary Chapman'], categories:['Family & Relationships'], review/score:5.0,title:The Happiest Baby on the Block: The New Way to Calm Crying and Help Your Newborn Baby Sleep Longer, authors:['Harvey Karp'], categories:['FAMILY & RELATIONSHIPS'], review/score:5.0"), 90.99234008789062), (Document(metadata={'user_id': 'A2BQIPOSD57QFX'}, page_content="title:The Art of Kissing, authors:['William Cane'], categories:['Family & Relationships'], review/score:4.0"), 154.9977569580078), (Document(metadata={'user_id': 'AZA2YLBR0I55P'}, page_content="title:Mrs. Fixit Easy Home Repair, authors:['Terri McGraw'], categories:['House & Home'], review/score:5.0"), 158.13145446777344), (Document(metadata={'user_id': 'A29LE714CCC8D2'}, page_content="title:The Art of Mingling: Proven Techniques for Mastering Any Room, authors:['Jeanne Martinet'], categories:['Self-Help'], revi

In [ ]:
def recommend_books(user_id, vectorstore, top_k=5):
    similar_users = find_similar_users(user_id, vectorstore, top_k)
    
    # Raccogliere i libri letti dagli utenti simili
    recommended_books = []
    user_books = vectorstore.similarity_search_with_score(f"User_id: {user_id}", k=1)[0][0].page_content
    
    for user in similar_users:
        similar_user_books = user[0].page_content
        # Filtra i libri che l'utente non ha ancora letto
        new_books = [book for book in similar_user_books if book not in user_books]
        recommended_books.extend(new_books)
    
    return set(recommended_books)  # Restituisci solo i libri unici

# Esempio di utilizzo
recommended_books = recommend_books(user_id, vectorstore)
print("Books recommended:", recommended_books)


In [8]:
#1 Metodo
query = "If i liked Economics in one lesson, What other books like it could you suggest to me?"
docs = recommend_db.similarity_search(query, k=5)
docs

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 11.53it/s]


[Document(metadata={'user_id': 5}, page_content="title:Economics in one lesson, description:With over a million copies sold, Economics in One Lesson is an essential guide to the basics of economic theory. A fundamental influence on modern libertarianism, Hazlitt defends capitalism and the free market from economic myths that persist to this day. Considered among the leading economic thinkers of the “Austrian School,” which includes Carl Menger, Ludwig von Mises, Friedrich (F.A.) Hayek, and others, Henry Hazlitt (1894-1993), was a libertarian philosopher, an economist, and a journalist. He was the founding vice-president of the Foundation for Economic Education and an early editor of The Freeman magazine, an influential libertarian publication. Hazlitt wrote Economics in One Lesson, his seminal work, in 1946. Concise and instructive, it is also deceptively prescient and far-reaching in its efforts to dissemble economic fallacies that are so prevalent they have almost become a new orthod

In [ ]:
from langchain.chains.retrieval_qa.base import RetrievalQA

#2 Metodo
qa = RetrievalQA.from_chain_type(llm=llm, chain_type = "stuff", retriever = recommend_db.as_retriever(), return_source_documents=True, verbose=True)

In [ ]:
query = "I'm looking for an drama book. What could you suggest to me?"
result = qa({"query": query})
result['result']

In [6]:
type(recommend_db)

langchain_community.vectorstores.chroma.Chroma

In [22]:
import re

# Stringa di input
data = "title:Wuthering Heights (New Windmill), authors:['Emily Brontë'], categories:['Foundlings'], review/score:5.0,title:Wuthering Heights, authors:['Emily Bronte'], categories:nan, review/score:5.0,title:Wuthering Heights, authors:['Emily Bronte'], categories:nan, review/score:5.0,title:Wuthering Heights, authors:['Emily Bronte'], categories:nan, review/score:5.0,title:Wuthering Heights, authors:['Emily Bronte'], categories:nan, review/score:5.0,title:Wuthering Heights, authors:['Emily Bronte'], categories:nan, review/score:5.0,title:Wuthering Heights, authors:['Emily Bronte'], categories:nan, review/score:5.0,title:Wuthering Heights, authors:['Emily Bronte'], categories:nan, review/score:5.0,title:Wuthering Heights, authors:['Emily Bronte'], categories:nan, review/score:5.0,title:Wuthering Heights (Riverside editions), authors:['Emily Brontë'], categories:nan, review/score:5.0,title:Wuthering Heights (Penguin Audiobooks), authors:nan, categories:nan, review/score:5.0,title:Wuthering Heights., authors:['Emily Bronte'], categories:nan, review/score:5.0,title:Wuthering Heights (Signet classics), authors:['Emily Brontë'], categories:['Audiobooks'], review/score:5.0,title:Wuthering Heights (Signet classics), authors:['Emily Brontë'], categories:['Audiobooks'], review/score:5.0,title:Ego and Archetype, authors:['Edward F. Edinger'], categories:['Psychology'], review/score:5.0,title:Wuthering Heights (College classics in English), authors:['Emily Brontë'], categories:['Fiction'], review/score:5.0,title:Jane Eyre / Wuthering Heights, authors:['Charlotte Brontë'], categories:['Boarding schools'], review/score:5.0,title:The Sunday Philosophy Club (Isabel Dalhousie Mysteries), authors:['Alexander McCall Smith'], categories:['Fiction'], review/score:5.0,title:THE PHANTOM OF THE OPERA, authors:['Gaston Leroux'], categories:['Fiction'], review/score:5.0,title:Wuthering Heights (The Franklin Library), authors:['Emily Brontë'], categories:nan, review/score:5.0,title:Fresh from the Vegetarian Slow Cooker: 200 Recipes for Healthy and Hearty One-Pot Meals That Are Ready When You Are, authors:['Robin Robertson'], categories:['Cooking'], review/score:5.0"

# Estrai i titoli usando un'espressione regolare
titles = re.findall(r'title:(.*?), authors', data)

# Mostra i risultati
print(type(titles))


<class 'list'>
